### Important
The code was done on an example dataset. Need to change to the real preprocessed data when available

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import os.path as op
from utils import mkdir_no_exist

import nibabel as nib
from nilearn.glm import threshold_stats_img
from nilearn.glm.first_level import make_first_level_design_matrix, FirstLevelModel
from nilearn.plotting import plot_design_matrix, plot_contrast_matrix, plot_stat_map
from nilearn.image import mean_img

from matplotlib import pyplot as plt

In [ ]:
# ACCESS DIRECTORIES #

current_dir = os.path.abspath("")
print(f"current_dir: {current_dir}")
sys.path.append(current_dir)

dataset_id = 'ds000171'
subjects = ['sub-control{:02d}'.format(i+1) for i in range(20)]

dataset_path = os.path.join(current_dir, "data", dataset_id)
deriv_path = os.path.join(current_dir,"data", "derivatives")
preproc_path = os.path.join(deriv_path, 'preprocessed_data')

mkdir_no_exist(dataset_path)
mkdir_no_exist(preproc_path)


In [ ]:
# READ AND CONCAT EVENTS #

events = {}
for i in range(1,4):
    events[i] = pd.read_csv(os.path.join(dataset_path,subjects[0],"func",f"sub-control01_task-music_run-{i}_events.tsv"), sep = "\t")
    if i>1: #adjust onset time for concatenation
        events[i]['onset'] += (events[i-1]['onset'].iloc[-1] + events[i-1]['duration'].iloc[-1])
        
# for key, event in events.items():
#     display(f"EVENT #{key}\n",event)
events_concat = pd.concat(events.values(),ignore_index=True)
display(events_concat)
events_concat.to_csv(op.join(deriv_path, 'events_concat.csv'))

In [ ]:
# ONLY FOR TESTING: need to replace with the preprocessed data
# plotted design matrix is not correct until glm is fit to own data!

from nilearn.datasets import fetch_spm_auditory
subject_data = fetch_spm_auditory()

from nilearn.image import concat_imgs, mean_img
fmri_img = concat_imgs(subject_data.func)

In [ ]:
task_music_bold = pd.read_json(op.join(dataset_path, "task-music_bold.json"), typ = 'series')

fmri_glm = FirstLevelModel(t_r = task_music_bold.RepetitionTime, 
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm + derivative',
                           drift_model="cosine",
                           high_pass=.01)

# Fit the model to our design and data
fmri_glm = fmri_glm.fit(fmri_img, events_concat)

# ONLY FOR TESTING: only has 80 scans bcs is fit against example fMRI,
# not the correct dataset
design = fmri_glm.design_matrices_[0]
plot_design_matrix(design,rescale=True, output_file = op.join(deriv_path, 'design_matrix.jpg'))
plt.show()

### Still to do for design matrix
- Discuss if add drift (I think yes)
- Discuss if add derivatives (and dispersions?) of regressors

  Yup it is justified since it captures better different shapes of empirical HRF

  Considerations on overfitting - i think we are very far from what i have understood

### Beta/ Statistical maps of each of the regressors

In [ ]:
def condition_vector(position:int, n_regressors:int)->np.array:
    vec = np.zeros((1,n_regressors))
    vec[0,position] = 1
    return vec

n_regressors = design.shape[1]
conditions = {
    col:condition_vector(idx, n_regressors) for idx, col in enumerate(design.columns)
}

In [ ]:
# All those that are not covariates
covariates = [key for key, value in conditions.items() if "drift" in key or "constant" in key]
conditions_of_interest = {key: value for key, value in conditions.items() if key not in covariates}

betas_zmap = op.join(preproc_path,'betas_zmap')
mkdir_no_exist(betas_zmap)

cluster_size = 10 #number of voxels for a cluster to be kept
fdr_rate = 0.05 # alpha = 5%

In [ ]:
#Plot the statistical map of a single component
key = 'negative_music'
value = conditions_of_interest[key]

z_map = fmri_glm.compute_contrast(value, output_type='z_score')
clean_map, threshold = threshold_stats_img(z_map, alpha=fdr_rate, height_control='fdr', cluster_threshold=cluster_size)

mean_img_ = mean_img(fmri_img),
plot_stat_map(clean_map, bg_img=mean_img_[0], threshold=threshold,
              display_mode='z', cut_coords=None, black_bg=True,
              title='Condition {} (fdr={}, threshold={}), clusters > {} voxels'.format(key, fdr_rate, threshold.round(2), cluster_size),
              #output_file = op.join(betas_zmap,f"z_map_fdr5p_cl10_{key}.jpg")
              )
plt.show()

In [ ]:
fig, axs = plt.subplots(len(conditions_of_interest), 1, figsize=(15,40))
for i, (key,value) in enumerate(conditions_of_interest.items()):
    z_map = fmri_glm.compute_contrast(value, output_type='z_score')
    
    # Apply the FDR correction + threshold
    clean_map, threshold = threshold_stats_img(z_map, alpha=fdr_rate, height_control='fdr', cluster_threshold=cluster_size)
    
    plot_stat_map(clean_map, bg_img=mean_img_[0], threshold=threshold,
                  display_mode='z', cut_coords=None, black_bg=True,
                  title='Condition {} (fdr={}, threshold={})'.format(key, fdr_rate, threshold.round(2)),
                  axes = axs[i])
fig.suptitle(f"Statistical maps of all regressors (FDR = {fdr_rate} and clusters > {cluster_size})")
plt.show()

### Contrasts

In [ ]:
positive_music_position = 2
negative_music_position = 0

pos_vs_neg = conditions['positive_music'] - conditions['negative_music']

plot_contrast_matrix(pos_vs_neg, design_matrix=design, output_file = op.join(deriv_path, 'contrast_pos_neg.jpg'))
plt.show()

In [ ]:
z_map = fmri_glm.compute_contrast(pos_vs_neg, output_type='z_score')
fdr_rate, cluster_size = 0.05, 10
clean_map, threshold = threshold_stats_img(z_map,
                                           alpha=fdr_rate,
                                           height_control='fdr',
                                           cluster_threshold=cluster_size)

mean_img_ = mean_img(fmri_img),
plot_stat_map(clean_map,
              bg_img=mean_img_[0],
              threshold=threshold,
              display_mode='z',
              cut_coords=[-3,20,36,70],
              black_bg=True,
              title='Positive vs. Negative Music (fdr={}, threshold={:.2f}, clusters > {} voxels)'.format(fdr_rate, threshold, cluster_size))
plt.show()